### Script translation text(en) to malagasy(mg)

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import torch
from google.colab import drive
from tqdm.auto import tqdm
import wandb
import argparse

import subprocess
import sys
import gdown


In [ ]:
# Définir les arguments pour le script
parser = argparse.ArgumentParser(description='Translate a portion of a CSV file using MADLAD-400-7B-MT-BT.')
parser.add_argument('--start_index', type=int, default=0, help='Index de la première ligne à traiter (inclusive).')
parser.add_argument('--end_index', type=int, default=10000, help='Index de la dernière ligne à traiter (exclusive).')
parser.add_argument('--batch_size', type=int, default=16, help='Taille du lot pour la traduction.')
parser.add_argument('--file_id', type=str, required=True, help='Id du fichier public sur drive')
args = parser.parse_args()

start_index = args.start_index
end_index = args.end_index
batch_size = args.batch_size
file_id = args.file_id

# Vérifier si un GPU est disponible et le définir comme device, sinon utiliser le CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Utilisation de l'appareil : {device}")

# modèle MADLAD-400-7B-MT-BT avec quantification 8 bits
model_name = "google/madlad400-7b-mt-bt"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True, device_map='auto')

# Télécharger le fichier CSV depuis l'URL Google Drive
local_file_name = f"downloaded_data_{file_id}.csv"
try:
    print(f"Téléchargement du fichier depuis {file_id}...")
    file_url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(file_url, local_file_name, quiet=False)
    file_path = local_file_name
    df = pd.read_csv(file_path)
    print("Fichier CSV téléchargé et chargé avec succès.")
except Exception as e:
    print(f"Erreur lors du téléchargement ou du chargement du fichier CSV depuis l'URL: {e}")
    exit()

# Vérifier que la colonne 'text' existe
if 'text' not in df.columns:
    print("Erreur: La colonne 'text' n'existe pas dans le fichier CSV.")
    exit()

# Sélectionner la portion du DataFrame à traiter
df_subset = df[start_index:end_index]
print(f"Traitement des lignes de {start_index} à {end_index - 1} par lots de {batch_size}.")

def translate_batch(texts):
    input_texts = [f"<2mg> {text}" for text in texts]
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            max_length=512
        )
    translated_texts = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return translated_texts

translations = [''] * len(df_subset)

# --- Configuration WandB
WANDB_API_KEY = "00623049297b4d6a9b3febd02306cf1294021a68"
os.environ["WANDB_API_KEY"] = WANDB_API_KEY # Définir la variable d'environnement pour WandB

# Initialiser WandB
wandb.init(project="hate-speech-translated-malagasy", name=f"translation-run-{start_index}-{end_index}")
# --- Fin de la configuration WandB ---


total_batches = len(df_subset) // batch_size + (1 if len(df_subset) % batch_size != 0 else 0)

# Appliquer la traduction par lots avec une barre de progression et suivi WandB
for i in tqdm(range(0, len(df_subset), batch_size), desc=f"Traduction ({start_index}-{end_index})"):
    batch = df_subset['text'][i:i + batch_size].tolist()
    translated_batch = translate_batch(batch)
    translations[i:i + batch_size] = translated_batch

    batch_number = i // batch_size + 1
    wandb.log({"progress": batch_number / total_batches * 100,
               "lines_processed": i + len(batch)})

# Ajouter la liste des traductions comme une nouvelle colonne au sous-ensemble
df_subset['translated'] = translations

# Créer un tableau WandB à partir du sous-ensemble traduit (contenant 'text', 'label' et 'translated')
wandb_table = wandb.Table(data=df_subset[['text', 'label', 'translated']], columns=["text", "label", "translated"])

# Enregistrer le tableau dans WandB
wandb.log({"translations_segment": wandb_table})

# Fermer la run WandB
wandb.finish()

In [ ]:
# %%bash
# #!/bin/bash

# echo "Début de l'exécution des traductions..."

# # Exemple pour traiter les lignes 0 à 999 avec un batch size de 8
# python run_translation.py --start_index 0 --end_index 1000 --batch_size 8 &

# # Exemple pour traiter les lignes 1000 à 1999 avec un batch size de 8
# python run_translation.py --start_index 1000 --end_index 2000 --batch_size 8 &

# wait # Attend que tous les processus en arrière-plan se terminent
# echo "Toutes les tâches de traduction sont terminées."

Process is interrupted.
